In [1]:
# Importation des différentes librairies utiles pour le notebook

#Sickit learn met régulièrement à jour des versions et 
#indique des futurs warnings. 
#ces deux lignes permettent de ne pas les afficher.
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pickle 
import pandas as pd
from scipy.stats import randint
import numpy as np
import string
import time
import base64
import re
import sys

import contractions

# librairie BeautifulSoup
from bs4 import BeautifulSoup

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud

## detection de language
import langdetect 

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import RegexpParser
# il est possible de charger l'ensemble des librairies en une seule fois 
# décocher le commentaire de la ligne ci-dessous
#nltk.download('all') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
nltk.download('tagsets')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords

import spacy
from spacy.tokens import Span
# il faut sélectionner pour quelle langue les traitements vont être faits.

# !python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
from spacy.lang.fr import French

[nltk_data] Downloading package punkt to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2023-03-13 10:00:38.049177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-crit

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [64]:
import pandas as pd
df = pd.read_csv('data/HAI817_Projet_train.csv')
df.head()



,public_id,text,title,our rating
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false


## Tokenization


In [65]:
def get_tokens(dataframe):    
    full_texts = []
    for i in range(len(dataframe)):
        full_texts.append((str(dataframe.iloc[i].title) + " " + str(dataframe.iloc[i].text)))
    tokens = [t.split() for t in full_texts]
    
    tokens = [[t.lower() for t in lt] for lt in tokens]
    return tokens




In [66]:
df["tokens"] = get_tokens(df)
df.head()

,public_id,text,title,our rating,tokens
0,5a228e0e,Distracted driving causes more deaths in Canad...,"You Can Be Fined $1,500 If Your Passenger Is U...",false,"[you, can, be, fined, $1,500, if, your, passen..."
1,30c605a1,Missouri politicians have made statements afte...,Missouri lawmakers condemn Las Vegas shooting,mixture,"[missouri, lawmakers, condemn, las, vegas, sho..."
2,c3dea290,Home Alone 2: Lost in New York is full of viol...,CBC Cuts Donald Trump's 'Home Alone 2' Cameo O...,mixture,"[cbc, cuts, donald, trump's, 'home, alone, 2',..."
3,f14e8eb6,But things took a turn for the worse when riot...,Obama’s Daughters Caught on Camera Burning US ...,false,"[obama’s, daughters, caught, on, camera, burni..."
4,faf024d6,It’s no secret that Epstein and Schiff share a...,Leaked Visitor Logs Reveal Schiff’s 78 Visits ...,false,"[leaked, visitor, logs, reveal, schiff’s, 78, ..."


### Eliminer stop words

In [67]:
nltk.download("stopwords")
from nltk.corpus import stopwords
the_stopwords=set(stopwords.words("english"))
the_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [68]:
print(f'avant : {df["tokens"].head()}')
df["tokens"] = df["tokens"].transform(lambda t: [word for word in t if word not in the_stopwords])
print(f'après : {df["tokens"].head()}')

avant : 0    [you, can, be, fined, $1,500, if, your, passen...
1    [missouri, lawmakers, condemn, las, vegas, sho...
2    [cbc, cuts, donald, trump's, 'home, alone, 2',...
3    [obama’s, daughters, caught, on, camera, burni...
4    [leaked, visitor, logs, reveal, schiff’s, 78, ...
Name: tokens, dtype: object
après : 0    [fined, $1,500, passenger, using, mobile, phon...
1    [missouri, lawmakers, condemn, las, vegas, sho...
2    [cbc, cuts, donald, trump's, 'home, alone, 2',...
3    [obama’s, daughters, caught, camera, burning, ...
4    [leaked, visitor, logs, reveal, schiff’s, 78, ...
Name: tokens, dtype: object


### Stemming

In [7]:
import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
ps=nltk.stem.porter.PorterStemmer()


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/e20220002252/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [61]:
df["token_stem"] = df["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])

print(df["token_stem"].head())



0    [you, can, be, fine, $1,500, if, your, passeng...
1    [missouri, lawmak, condemn, la, vega, shoot, m...
2    [cbc, cut, donald, trump', 'home, alon, 2', ca...
3    [obama’, daughter, caught, camera, burn, us, f...
4    [leak, visitor, log, reveal, schiff’, 78, visi...
Name: token_stem, dtype: object


In [9]:
corpus = df["token_stem"].transform(lambda s: " ".join(s)).tolist()


## Bag of words


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
#premier exemple sans paramètre


# par defaut conversion en minuscule
vectorizer = CountVectorizer(stop_words=list(set(stopwords.words("english"))))

# creation du vocabulaire
vectorizer.fit(corpus)
# encodage du document
vector = vectorizer.transform(corpus)

# la liste des différents features
print ("Les différents features sont",vectorizer.get_feature_names_out(),' ... à noter tout est converti en minuscule\n')

# Contenu du vocabulaire
print ("Vocabulaire : ")
print(vectorizer.vocabulary_)


# affichage de la taille du vecteur de sortie

print ("\nTaille du vecteur :\n",vector.shape,'\n')

Les différents features sont ['00' '000' '0001' ... 'ввп' 'победа' 'привет']  ... à noter tout est converti en minuscule

Vocabulaire : 
{'fine': 10733, '500': 1069, 'passeng': 19732, 'use': 27881, 'mobil': 17499, 'phone': 20176, 'start': 25029, 'next': 18379, 'week': 28669, 'distract': 8687, 'drive': 8994, 'caus': 5426, 'death': 7739, 'canada': 5120, 'impair': 13609, 'everi': 10027, 'provinc': 21247, 'territori': 26229, 'law': 15537, 'oper': 19064, 'cell': 5495, 'tell': 26139, 'stay': 25080, 'driving': 9001, 'measur': 16880, 'necessari': 18202, 'claim': 6009, 'live': 15974, 'like': 15865, 'british': 4683, 'columbia': 6358, 'ontario': 19045, 'quebec': 21485, 'alberta': 2275, 'nova': 18642, 'scotia': 23478, 'manitoba': 16516, 'newfoundland': 18346, 'labrador': 15354, 'phones': 20177, 'even': 10013, 'held': 12732, 'passenger': 19733, 'danger': 7618, 'driver': 8997, 'screen': 23495, 'attract': 3254, 'penalti': 19926, 'three': 26438, 'demerit': 7999, 'points': 20492, 'mix': 17475, 'matter'

In [11]:
df2 = pd.DataFrame(
    data=vectorizer.transform(corpus).toarray(),
    columns=vectorizer.get_feature_names_out()
)

display(df2)

,00,000,0001,0008,000m,000x,001,0014,002,0020,...,zzt2briay,µg,µm2,ärzteblatt,örjan,único,μm,ввп,победа,привет
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1261,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df2.insert(0, "our rating", df["our rating"])


In [13]:
display(df2)

,our rating,00,000,0001,0008,000m,000x,001,0014,002,...,zzt2briay,µg,µm2,ärzteblatt,örjan,único,μm,ввп,победа,привет
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,mixture,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,mixture,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1260,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1261,false,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1262,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model


In [14]:
import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [30]:
values = df2.values
X = values[:, 1:]
y = values[:, 0]

clf = GaussianNB()
clf.fit(X, y)

res = clf.predict(X)
print(f'accuracy = {accuracy_score(res, y)}')

accuracy = 0.9778481012658228


### test avec 2 classes

In [36]:
df_binaire = df2[(df2["our rating"] == 'true') | (df2["our rating"] == 'false')]
display(df_binaire)

,our rating,00,000,0001,0008,000m,000x,001,0014,002,...,zzt2briay,µg,µm2,ärzteblatt,örjan,único,μm,ввп,победа,привет
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1260,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1261,false,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1262,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
test_data = pd.read_csv('data/HAI817_Projet_test.csv')
test_data["tokens"] = get_tokens(test_data)
test_data["tokens"].transform(lambda t: [word for word in t if word not in the_stopwords])
test_data["token_stem"] = test_data["tokens"].transform(lambda tokens: [ps.stem(word) for word in tokens])
corpus_test = test_data["token_stem"].transform(lambda s: " ".join(s)).tolist()

In [16]:


# par defaut conversion en minuscule
vectorizer2 = CountVectorizer(stop_words=list(set(stopwords.words("english"))))
# creation du vocabulaire
vectorizer2.fit(corpus_test)
# encodage du document
vector2 = vectorizer2.transform(corpus_test)

test_df = pd.DataFrame(
    data=vectorizer2.transform(corpus_test).toarray(),
    columns=vectorizer2.get_feature_names_out()
)

test_df.insert(0, "our rating", test_data["our rating"])
display(test_df)



,our rating,00,000,00017,00054,0008,000km,000mile,001,00106,...,ראשון,רוב,שבויים,שבת,שהשחיתות,שחיתות,שישי,של,שלישי,תיגר
0,mixture,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,other,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,other,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
609,true,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
610,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


ValueError: X has 23397 features, but GaussianNB is expecting 14739 features as input.

In [ ]:
vocab_train = list(vectorizer.vocabulary_.keys())
vocab_test = list(vectorizer2.vocabulary_.keys())

inter = [mot for mot in vocab_train if mot in vocab_test]
train_drop = [mot for mot in vocab_train if mot not in inter]
test_drop = [mot for mot in vocab_test if mot not in inter]

print(len(inter))
print(len(train_drop))
print(len(test_drop))

### dataframe train et test contenant seulement les mots en commun


In [48]:
df_train = df2.drop(train_drop, 1)
df_train_bin = df_train[(df_train["our rating"] == 'true') | (df_train["our rating"] == 'false')]
df_train_tri = df_train[(df_train["our rating"] == 'true') | (df_train["our rating"] == 'false') | (df_train["our rating"] == 'other')]

df_test = test_df.drop(test_drop, 1)
df_test_bin = df_test[(df_test["our rating"] == 'true') | (df_test["our rating"] == 'false')]
df_test_tri = df_test[(df_test["our rating"] == 'true') | (df_test["our rating"] == 'false') | (df_test["our rating"] == 'other')]

display(df_train_tri)
display(df_test_tri)

,our rating,00,000,0008,001,002,003,005,00am,00pm,...,zionist,zip,zombi,zone,zones,zoom,zoonot,zucker,zuckerberg,zurich
0,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,other,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1260,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1261,false,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1262,true,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,our rating,00,000,0008,001,002,003,005,00am,00pm,...,zionist,zip,zombi,zone,zones,zoom,zoonot,zucker,zuckerberg,zurich
1,other,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,false,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,other,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
609,true,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
610,false,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
clf = GaussianNB()
X = df_train_bin.values[:, 1:]
y = df_train_bin.values[:, 0]
clf.fit(X, y)

X_test = df_test_bin.values[:, 1:]
y_test = df_test_bin.values[:, 0]
res = clf.predict(X_test)
print(f'accuracy = {accuracy_score(res, y_test)}')


accuracy = 0.6228571428571429


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.model_selection import cross_val_score

 ## Test d'accuracy sur les données d'entraînement avec chaque classifieur

In [52]:

models = []
models.append(('LR', LogisticRegression()))
# models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
# models.append(('SVC', SVC(gamma='auto')))
models.append(('LSVC', LinearSVC(max_iter=3000)))
models.append(('RFC', RandomForestClassifier()))
# models.append(('DTR', DecisionTreeRegressor()))

for name,model in models:
    model.fit(X, y)
    res = model.predict(X)
    acc = accuracy_score(res, y)
    names.append(name)
    msg = "%s: acc = %f" % (name, acc)
    print(msg) 

/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR: acc = 0.998733
NB: acc = 0.970849


/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LSVC: acc = 0.997465
RFC: acc = 0.998733


In [60]:
models = []
models.append(('LR', LogisticRegression()))
# models.append(('KNN', KNeighborsClassifier()))
models.append(('NB', GaussianNB()))
# models.append(('SVC', SVC(gamma='auto')))
models.append(('LSVC', LinearSVC(max_iter=3000)))
models.append(('RFC', RandomForestClassifier()))
# models.append(('DTR', DecisionTreeRegressor()))
results = []
names = []
scoring='accuracy'
for name,model in models:
    kfold = KFold(n_splits=10, random_state=7,shuffle=True)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring, n_jobs=-1)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg) 


/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

LR: 0.830169 (0.036751)
NB: 0.783285 (0.057798)


/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/e20220002252/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1244

LSVC: 0.807319 (0.046755)
RFC: 0.814963 (0.056922)
